In [2]:
import os, sys, json, re, xlrd  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
from datetime import date
import pandas as pd             # Provides data structures and data analysis tools
from openpyxl import Workbook
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import glob
import time
from tqdm import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown

from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, add_seats, get_key, get_recent_file
from notebook.services.config import ConfigManager



import os, sys, json, datetime, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
from tqdm import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
from lxml import etree, html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys


## Process

- get legislator info
    - *done* pull in data from quorum using quorum_officials_manipulation.py
        - gets files for each state
    - *done* run all_legs_file_creation.py
        - organizes and cleans data to ideal setup
        - create df's for use in influence calculation
        - outputs
            - all_leg
            - dfs_w_pk's
            - all_leg_records
    - *done* run and create keys with key_creation.py
        - use all_leg files for the year (from all_legs_file_creation)
        - outputs
            - key_lookup
- Influence Calculation
    - Pull data
        - Pull key_lookup (from key_creation)
        - Manual Leadership File
        - dfs_w_pk's (from all_legs_creation)
    - Calculate Influence Score
        - leg_infl_df[date].csv
    - Create Priority Analysis file
        - comms_summary.csv
- Activity Calculation
    - Pulls in data
        - Leg_lookup (from key_creation)
        - Attendance data (attendance data folder)
    - Outputs: 
        - activities_scores[date].csv
        - event_summary[date].csv
- Compiling Scores *i need to split up the compiled plus bills and bills and legislators into their own script*
    - Main compilation
        - Input Files:
            - activities_scores[date].csv
            - leg_infl_df[date].csv
            - leg_lookup
        - Exports:
            - compiled_scores (final_df)
    - Compiled plus bills *needs bill pull ran first*
        - Inputs
            - legislator info (from bill pulling)
            - Final_df (local)
        - Outputs:
            - compiled_plus_bills.csv
            - bills_and_legislators.csv
- Quorum Bill Pulling
        - *needs to be updated regularly*
        - Gets bills from quorum bill_downloads (..\bill_data\quorum downloads)
        - Outputs:
            - ed_bills (local)
            - legislator_info.csv
    - Final join
        - inputs:
            - ed_bills
            - bills_and_legislators





## Excution

### Pulling Data From Quorum

In [78]:
# Pull files from quorum

webdriver_path = r"C:\Users\clutz\hunt_env\chrome driver\chromedriver-win64\chromedriver.exe"
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
# Set up WebDriver service
service = Service(webdriver_path)

#call on driver and get data
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)


In [ ]:
#log in
page_url = r'https://www.quorum.us/sheets/'
driver.get(page_url)

username_field = driver.find_element(By.ID,"username")
# login_form = driver.find_element(By.ID, 'loginForm')
username_field.click()
username_field.send_keys('clutz@hunt-institute.org')

password_field = driver.find_element(By.ID, "password");
password_field.click()
password_field.send_keys('4Ducksquack!')
password_field.send_keys('\ue006')
time.sleep(10)


In [88]:
#get sheet links
page_url = r'https://www.quorum.us/sheets/'
driver.get(page_url)

html_from_page = driver.page_source
# print(html_from_page)
soup = BeautifulSoup(html_from_page, 'html.parser')



links = soup.find_all("a", {"class":"datum-link"},href=True)
main_url = page_url.split('sheets')[0].strip()
sheet_links = {}
for l in links:
    # print(l)
    # print(type(l))
    new_l = str(l).split('public_fixedDataTableCell_cellContent">', 1)[-1].split('</')[0].strip()
    # print(new_l)
    sheet_link = f"{main_url}{str(l).split('href="', 1)[-1].split('">')[0].lstrip('/')}"


    if re.search(r'_thii', str(new_l)):
        sheet_links[new_l] = sheet_link
    elif re.search('elected_officials', str(new_l)):
        sheet_links[new_l] = sheet_link

In [89]:
for k,v in sheet_links.items():
    print(k)
    print(v)

ND Bills_thii
https://www.quorum.us/sheet/126248/
2025_thi_states_bills_thii
https://www.quorum.us/sheet/124234/
2025_non_thi_states_bills_thii
https://www.quorum.us/sheet/124235/
elected_officials
https://www.quorum.us/sheet/128847/


In [ ]:
#loop and download
#they should go to the right folder

for k,v in sheet_links:
    time.sleep(60)
    driver.get(v)
    dl_button = driver.find_element(By.ID,"download-sheet")
    dl_button.click()

    excel_button = driver.find_element(By.XPATH, '/html/body/div[85]/div/div[4]/div[1]/button')
    excel_button.click()
driver.quit()

# Running Scripts

In [13]:
print(os.listdir())  # See if the file is present


['all_legs_file_creation.ipynb', 'attendance', 'compiling_act_and_infl_scores.ipynb', 'event_data_stats.ipynb', 'execution.ipynb', 'get_activities_score.ipynb', 'get_influence_score.ipynb', 'influence_maintenance.ipynb', 'influence_maintenance.md', 'key_creation.ipynb', 'merge_cohorts_and_activities.ipynb']


In [ ]:
#run all_legs_file_creation
import subprocess
import sys


run_process = subprocess.run(["jupyter", "nbconvert", "--to", "notebook", "--execute", "all_legs_file_creation.ipynb"],    
                             capture_output=True, text=True
)

# Check if the script execution was successful
if run_process.returncode != 0:
    print("Error while running all_legs_file_creation.py:")
    print(run_process.stderr)  # Print script error output
    exit(1)



In [ ]:
#running key_creation
import subprocess
import sys


run_process = subprocess.run(["jupyter", "nbconvert", "--to", "notebook", "--execute", "key_creation.ipynb"],    
                             capture_output=True, text=True
)

# Check if the script execution was successful
if run_process.returncode != 0:
    print("Error while running key_creation.py:")
    print(run_process.stderr)  # Print script error output
    exit(1)


: 